## Loading Packages & Initialization 

In [1]:
rm(list=ls())

library(data.table)
library(tidyverse)
library(rJava)
library(RNetLogo)
library(lhs)

options(warn = -1)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1     ✔ purrr   0.3.2
✔ tibble  3.1.0     ✔ dplyr   1.0.5
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
Warning message:
“package ‘tibble’ was built under R version 3.6.3”Warning message:
“package ‘tidyr’ was built under R version 3.6.3”Warning message:
“package ‘dplyr’ was built under R version 3.6.3”── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()   masks data.table::between()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::first()     masks data.table::first()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::last()      masks data.table::last()
✖ purrr::transpose() masks data.tab

In [2]:
folder.path = "/Users/ecemnaz.yildiz/Documents/Personal/Thesis/"
source(paste0(folder.path,"ThesisSetupCode.r"))

Is_Headless <- 1
nl.model <- "info_cascade_update_TDP_JPF_2020"

nl.path <- "/Users/ecemnaz.yildiz/Documents/NetLogo 6.0.4/Java"
folder.path = "/Users/ecemnaz.yildiz/Documents/Personal/Thesis/"

model.path <- paste0(folder.path, nl.model, ".nlogo")

if (Is_Headless == 0) {
    NLStart(nl.path, gui = TRUE, nl.jarname = "netlogo-6.0.4.jar")
    NLLoadModel(model.path)
} else {
    NLStart(nl.path, gui = FALSE, nl.jarname = "netlogo-6.0.4.jar", nl.obj = nl.model)
    NLLoadModel(model.path, nl.obj = nl.model)
}

options(warn = -1)


Attaching package: ‘plotly’

The following object is masked from ‘package:igraph’:

    groups

The following object is masked from ‘package:ggplot2’:

    last_plot

The following object is masked from ‘package:stats’:

    filter

The following object is masked from ‘package:graphics’:

    layout

Loading required package: lattice

Attaching package: ‘caret’

The following object is masked from ‘package:purrr’:

    lift

randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:dplyr’:

    combine

The following object is masked from ‘package:ggplot2’:

    margin

Welcome! Want to learn more? See two factoextra-related books at https://goo.gl/ve3WBa
Loading required package: gridExtra

Attaching package: ‘gridExtra’

The following object is masked from ‘package:randomForest’:

    combine

The following object is masked from ‘package:dplyr’:

    combine



In [3]:
model.type = "info_cascade_update"
# the path of data folder
data.path = paste0(folder.path,"data/")
# the path for outputs to be record
output.folder = paste0("outputs_AdS_cvar_TimeTrack_",model.type,"_",Sys.Date())
dir.create(file.path(folder.path, output.folder), showWarnings = FALSE)

outputs.path = paste0(folder.path,output.folder,"/")

# Read Me File to keep info about the output folder
ReadMe = paste0(outputs.path,"ReadMe_",model.type,".txt")

## Model Parameters & Functions

### Set model parameters

In [4]:
#### Model Parameters ####
## Set model parameters Number of replications for each instance
nofrep = 30

feature_names = c("ground-truth","max_links"
                  ,"evidence","sc-bel-prop"
                  ,"prop-likelihood","p-h-given-c"
                  ,"n_init_believers","learning_rate"
                  ,"con-threshold","expertise_influence") 
    
feature_ranges = data.table(  feature   = feature_names
                            , min_range = c(0, 0, 0, 0, 0, 0, 0, 0, 0)
                            , max_range = c(1, 500, 100, 5, 1, 1, 100, 1, 1, 1)
                           )

output_name = c("cl-prop-same")

# Number of input parameters of the agent-based model
nofparams = length(feature_names)

# set RF parameters
ntree = 300
mtry = 6
nperm = 5

### Set user parameters

In [6]:
#### User parameters ####
error_type = "RMSE"  # MAPE, BIAS

# choose the uncertainty measure
selection_metric <- "coefvar"  #, 'range' 
sample.type = selection_metric

# Number of iterations
iteration_budget = 11
metarep = c(1:1)

# Number of instances
unlabeled_ins = 30
test_ins = c(30) #c(100,200,300,400)
train_ins_oneshot = 30
train_ins_Ad = 30

# Set selection parameters
selected_ins = 5  #nofinstancesWillbeSelected in each step

# Set elimination parameters
h <- 1  # number of variables eliminated in each step

seed.focus = c(0)#,1,2,3,4,5,6,7,8,9,20)

## to be used in log entries
unlabeled.type = "refresh"

log_entry.sampling()

## Test Set

In [10]:
#### Test Sets ####
test_set = data.table()

for( t in test_ins){
    test_set.name= paste0(data.path,"test_set","_",model.type,"_",t,".csv")
    test_set_Sub <- fread(test_set.name)  
    
    test_set = rbind(test_set, data.table(size = t, test_set_Sub))
}

In [11]:
test_set

size,ground-truth,max_links,evidence,sc-bel-prop,prop-likelihood,p-h-given-c,n_init_believers,learning_rate,con-threshold,expertise_influence,output
30,0.3042768,89.112113,70.470940,4.9971487,0.21415365,0.31318558,73.333450,0.388069845,0.25251921,0.68713367,43.63463
30,0.4747389,129.149248,33.784110,1.0882790,0.66338854,0.69404958,26.549282,0.529586065,0.55243245,0.21574479,47.60641
30,0.9935258,402.767907,10.233580,2.1787607,0.92243732,0.20535621,49.550760,0.549731941,0.36439035,0.21913514,45.52467
30,0.5206539,82.418144,66.200631,0.2948646,0.15308214,0.89228352,50.357083,0.239288878,0.58056254,0.92578717,45.60589
30,0.8432310,488.704130,73.908265,3.1535899,0.43870431,0.58688183,67.234487,0.782341848,0.33147896,0.78232558,44.08580
30,0.7233145,17.862386,89.873140,4.4003518,0.99618287,0.88094505,43.866759,0.068812180,0.60581744,0.75735539,52.11613
30,0.6151681,412.752611,29.606159,0.5963210,0.44761687,0.98306526,41.839219,0.190624533,0.02576714,0.07967662,46.26020
30,0.7391713,20.572940,52.469591,1.7078387,0.81519827,0.32036658,45.222689,0.043802018,0.29116257,0.71565510,50.73674
30,0.4172400,26.537910,15.394895,4.2969741,0.18812904,0.81556064,66.006938,0.123153712,0.20104112,0.96558474,45.73909
30,0.3726250,256.745520,66.876126,3.0851056,0.20374490,0.59864803,20.708671,0.377884488,0.17905255,0.75177780,48.11829


## One-Shot Training Set

In [14]:
one_shot_data = upload_training_set(model.type,seed.focus,train_ins_oneshot)

In [15]:
one_shot_data

ground-truth,max_links,evidence,sc-bel-prop,prop-likelihood,p-h-given-c,n_init_believers,learning_rate,con-threshold,expertise_influence,output,seed
0.47601769,309.80882,83.549839,2.68861870,0.65168982,0.63354368,57.8611225,0.54632148,0.32160454,0.541202792,44.91015,0
0.35672834,317.30518,63.106664,2.22335191,0.90763209,0.56252703,76.1547683,0.47592333,0.46328014,0.524350752,43.35827,0
0.71638853,80.00353,58.991850,2.40930323,0.68969464,0.74683716,43.1984348,0.50098188,0.38406676,0.475940387,46.13721,0
0.38894489,268.21950,83.062600,3.66248914,0.62880975,0.60311785,34.5217967,0.23294625,0.70413467,0.239238373,46.87023,0
0.50152214,376.17234,68.169001,2.96416124,0.50981017,0.86337559,16.2605515,0.77409926,0.60234462,0.610609240,48.47599,0
0.40541209,263.49302,47.911606,1.78630814,0.26884695,0.45373039,77.0902000,0.60907955,0.67106358,0.441360433,43.27983,0
0.44819153,86.80230,95.770899,3.07717965,0.46432791,0.26671911,96.6376782,0.96163352,0.80923608,0.887869010,41.82674,0
0.53651635,214.02388,63.358828,3.30239302,0.18843268,0.47324912,90.6509697,0.24464914,0.13135634,0.795441728,42.26746,0
0.07952148,217.82492,33.549733,0.98303740,0.78358728,0.14660758,31.4393862,0.44938462,0.48563582,0.428534337,47.14763,0
0.24501809,50.51225,74.346076,1.62021644,0.43220176,0.89463243,12.5805322,0.34182141,0.54708903,0.205643966,51.20797,0


### One-shot Train & Test Metamodel

In [16]:
sample.type = "oneshot"

sample.folder = paste0(sample.type,"/")
dir.create(file.path(folder.path, output.folder,sample.folder), showWarnings = FALSE)

models.folder = paste0("models_",sample.type,"/")
dir.create(file.path(folder.path, output.folder,models.folder), showWarnings = FALSE)

PL.folder = paste0("PL_",sample.type,"/")
dir.create(file.path(folder.path, output.folder,PL.folder), showWarnings = FALSE)

iter = 1
for (i in seed.focus) {
    
    training_set = copy(one_shot_data[seed == i, .SD, .SDcols = -c("seed")])
    for (r in metarep) {
        set.seed(i + r)
        run_log_entry()

        model_Sub <- randomForest(x = training_set[, -c("output")], y = training_set$output, importance = TRUE, ntree = ntree, mtry = mtry, nperm = nperm)      
        model_Sub.path = paste0(outputs.path,models.folder,"model_",sample.type,"_seed_", i, "_rep_", r,"_size_",train_ins_oneshot,".rds")
        saveRDS(model_Sub, model_Sub.path)
        
        # write errors 
        obb_err = obb_error_func(model_Sub)     
        fwrite(data.table(iter,obb_error = obb_err,seed = i,rep = r)
            ,paste0(outputs.path,sample.folder,model.type,"_","obb_error_",sample.type,".csv") ,append = TRUE)
        
        write_test_accuracy(i,r,iter,model_Sub,test_set, error_type)
        write_importance.rf(i,r,iter,model_Sub,sample.type)
        
    }
}

### Adaptive Training Set

In [17]:
adaptive_initial_data = upload_training_set(model.type,seed.focus,train_ins_Ad)

In [18]:
adaptive_initial_data

ground-truth,max_links,evidence,sc-bel-prop,prop-likelihood,p-h-given-c,n_init_believers,learning_rate,con-threshold,expertise_influence,output,seed
0.47601769,309.80882,83.549839,2.68861870,0.65168982,0.63354368,57.8611225,0.54632148,0.32160454,0.541202792,44.91015,0
0.35672834,317.30518,63.106664,2.22335191,0.90763209,0.56252703,76.1547683,0.47592333,0.46328014,0.524350752,43.35827,0
0.71638853,80.00353,58.991850,2.40930323,0.68969464,0.74683716,43.1984348,0.50098188,0.38406676,0.475940387,46.13721,0
0.38894489,268.21950,83.062600,3.66248914,0.62880975,0.60311785,34.5217967,0.23294625,0.70413467,0.239238373,46.87023,0
0.50152214,376.17234,68.169001,2.96416124,0.50981017,0.86337559,16.2605515,0.77409926,0.60234462,0.610609240,48.47599,0
0.40541209,263.49302,47.911606,1.78630814,0.26884695,0.45373039,77.0902000,0.60907955,0.67106358,0.441360433,43.27983,0
0.44819153,86.80230,95.770899,3.07717965,0.46432791,0.26671911,96.6376782,0.96163352,0.80923608,0.887869010,41.82674,0
0.53651635,214.02388,63.358828,3.30239302,0.18843268,0.47324912,90.6509697,0.24464914,0.13135634,0.795441728,42.26746,0
0.07952148,217.82492,33.549733,0.98303740,0.78358728,0.14660758,31.4393862,0.44938462,0.48563582,0.428534337,47.14763,0
0.24501809,50.51225,74.346076,1.62021644,0.43220176,0.89463243,12.5805322,0.34182141,0.54708903,0.205643966,51.20797,0


### Random Sampling Train & Test Metamodel

In [19]:
sample.type = "Rd"
sample.folder = paste0(sample.type,"/")
dir.create(file.path(folder.path, output.folder,sample.folder), showWarnings = FALSE)

models.folder = paste0("models_",sample.type,"/")
dir.create(file.path(folder.path, output.folder,models.folder), showWarnings = FALSE)

PL.folder = paste0("PL_",sample.type,"/")
dir.create(file.path(folder.path, output.folder,PL.folder), showWarnings = FALSE)

for (i in seed.focus) {
    print(paste0("seed : ", i, "  Random Sampling section start time : ", Sys.time()))
    
    for (r in metarep) { #replications
        set.seed(i + r) # set seed to control randomness
        print(paste0("seed : ", i,"   rep : ", r, "  Random Sampling section start time : ", Sys.time()))
        
        training_set_Ad = copy(adaptive_initial_data[seed == i, .SD, .SDcols = -c("seed")])       
        train_candidates_table = data.table()
            
        iter = 1
        while(iter <= iteration_budget){
            print(iter)
            run_log_entry()
            
            trainx = training_set_Ad[, .SD, .SDcols = feature_names]
            trainy = training_set_Ad$output
            
            # Train the model
            model_Sub <- randomForest(x = trainx, y = trainy, importance = TRUE,ntree = ntree, mtry = mtry, nperm = nperm)
            model_Sub.name = paste0("model_",sample.type,"_", iter, "_seed_", i, "_rep_",r)
            assign(model_Sub.name, model_Sub)
            model_Sub.path = paste0(outputs.path,models.folder, paste0(model_Sub.name,"_size_",train_ins_Ad, ".rds"))  # to save the model
                saveRDS(model_Sub, model_Sub.path)
            
            # write errors 
            obb_err = obb_error_func(model_Sub)     
            fwrite(data.table(iter,obb_error = obb_err,seed = i,rep = r)
                   ,paste0(outputs.path,sample.folder,model.type,"_","obb_error_",sample.type,".csv") ,append = TRUE)
            
            write_test_accuracy(i,r,iter,model_Sub,test_set, error_type)
            write_importance.rf(i,r,iter,model_Sub,sample.type)#last one=sample_type

                   
            if (iter != iteration_budget) {
                ## sample selection from unlabeled data select candidates
                 
                unlabeled_set <- refresh_sample_pool(i + r + iter)
                train_candidates = random_sample_selection(selected_ins, unlabeled_set)
                                
                # run ABM to find outputs of train candidates
                print(paste0("ABM train_candidate run start time : ", Sys.time()))
                train_candidates = run_ABM(nofrep, selected_ins, train_candidates)
                print(paste0("ABM train_candidate run end time : ", Sys.time()))
                
                
                fwrite(data.table(train_candidates, "iter" = iter, "seed" = i, "rep" = r)
                       ,paste0(outputs.path,sample.folder,model.type,"_train_candidates_table_",sample.type,".csv"),append = TRUE )               
 
                # Add new data to train data
                training_set_Ad = rbind(training_set_Ad, train_candidates[, -c("idx")],use.names = TRUE)
            }
            iter = iter + 1
        }  
        
        fwrite(data.table(training_set_Ad, "seed" = i,"rep" = r),paste0(outputs.path,sample.folder,model.type,"_FinalTrainData_",sample.type,".csv") ,append = TRUE)
    
        print(paste0("seed : ", i,"   rep : ", r, "  Random Sampling section end time : ", Sys.time()))        
    }  
         
    print(paste0("seed : ", i, "  Random Sampling section end time : ", Sys.time()))
}

[1] "seed : 0  Random Sampling section start time : 2021-11-14 22:52:09"
[1] "seed : 0   rep : 1  Random Sampling section start time : 2021-11-14 22:52:09"
[1] 1
[1] "ABM train_candidate run start time : 2021-11-14 22:52:09"
[1] "ABM train_candidate run end time : 2021-11-14 22:56:03"
[1] 2
[1] "ABM train_candidate run start time : 2021-11-14 22:56:03"
[1] "ABM train_candidate run end time : 2021-11-14 22:59:12"
[1] 3
[1] "ABM train_candidate run start time : 2021-11-14 22:59:12"
[1] "ABM train_candidate run end time : 2021-11-14 23:01:40"
[1] 4
[1] "ABM train_candidate run start time : 2021-11-14 23:01:40"
[1] "ABM train_candidate run end time : 2021-11-14 23:04:20"
[1] 5
[1] "ABM train_candidate run start time : 2021-11-14 23:04:20"
[1] "ABM train_candidate run end time : 2021-11-14 23:06:57"
[1] 6
[1] "ABM train_candidate run start time : 2021-11-14 23:06:58"
[1] "ABM train_candidate run end time : 2021-11-14 23:10:00"
[1] 7
[1] "ABM train_candidate run start time : 2021-11-14 23:10

### Adaptive Train & Test Metamodel

In [20]:
adaptive_initial_data = upload_training_set(model.type,seed.focus,train_ins_Ad)

In [21]:
sample.type = paste0("Ad_",selection_metric)
sample.folder = paste0(sample.type,"/")
dir.create(file.path(folder.path, output.folder,sample.folder), showWarnings = FALSE)

models.folder = paste0("models_",sample.type,"/")
dir.create(file.path(folder.path, output.folder,models.folder), showWarnings = FALSE)

PL.folder = paste0("PL_",sample.type,"/")
dir.create(file.path(folder.path, output.folder,PL.folder), showWarnings = FALSE)

for(i in seed.focus){

    print(paste0("seed : ",i,"  Adaptive Sampling with ",selection_metric,"  section start time : ",Sys.time()))
    
    for (r in metarep){ #replications
        set.seed(i + r)
        print(paste0("seed : ", i,"   rep : ", r, "  Adaptive Sampling section start time : ", Sys.time()))
            
        training_set_Ad = copy(adaptive_initial_data[seed == i, .SD, .SDcols = -c("seed")])
        train_candidates_table = data.table()

        iter = 1
        while(iter <= iteration_budget){   
            print(iter)
            run_log_entry() 
        
            trainx = training_set_Ad[,.SD, .SDcols = feature_names]
            trainy = training_set_Ad$output
        
            # Train the model
            model_Sub <- randomForest( x = trainx, y =  trainy,importance = TRUE,ntree = ntree, mtry = mtry,nperm = nperm)
            model_Sub.name = paste0("model_",sample.type,"_", iter, "_seed_", i, "_rep_",r)
            model_Sub.path = paste0(outputs.path,models.folder, paste0(model_Sub.name,"_size_",train_ins_Ad, ".rds"))  # to save the model
            saveRDS(model_Sub, model_Sub.path)
        
            # write errors 
            obb_err = obb_error_func(model_Sub)     
            fwrite(data.table(iter,obb_error = obb_err,seed = i,rep = r)
                   ,paste0(outputs.path,sample.folder,model.type,"_","obb_error_",sample.type,".csv") ,append = TRUE)
        
            write_test_accuracy(i,r,iter,model_Sub,test_set, error_type)
            write_importance.rf(i,r,iter,model_Sub,sample.type)#last one=sample_type
        
            if(iter != iteration_budget){ # below efforts are unnecessary when the budget is reached.    
                ## sample selection from unlabeled data select candidates
                unlabeled_set <- refresh_sample_pool(i + r + iter)
            
                train_candidates = sample_selection(selected_ins, unlabeled_set, model_Sub,selection_metric)
            
                # run ABM to find outputs of train candidates
                print(paste0("ABM train_candidate run start time : ",Sys.time()))
                train_candidates = run_ABM(nofrep, selected_ins, train_candidates)
                print(paste0("ABM train_candidate run end time : ",Sys.time()))
            
                fwrite(data.table(train_candidates, "iter" = iter, "seed" = i, "rep" = r)
                       ,paste0(outputs.path,sample.folder,model.type,"_train_candidates_table_",sample.type,".csv"),append = TRUE )      

                # add labeled candidates to the train data
                training_set_Ad = rbind(training_set_Ad, train_candidates[, -c("idx")],use.names = TRUE)
            }  
        
            iter = iter + 1
        }
        fwrite(data.table(training_set_Ad, "seed" = i,"rep" = r),paste0(outputs.path,sample.folder,model.type,"_FinalTrainData_",sample.type,".csv") ,append = TRUE)
    
        print(paste0("seed : ",i,"   rep : ", r,"  Adaptive Sampling section end time : ",Sys.time()))
    }
        
    print(paste0("seed : ",i,"  Adaptive Sampling section end time : ",Sys.time()))
}

[1] "seed : 0  Adaptive Sampling with coefvar  section start time : 2021-11-14 23:21:29"
[1] "seed : 0   rep : 1  Adaptive Sampling section start time : 2021-11-14 23:21:29"
[1] 1
[1] "ABM train_candidate run start time : 2021-11-14 23:21:29"
[1] "ABM train_candidate run end time : 2021-11-14 23:25:57"
[1] 2
[1] "ABM train_candidate run start time : 2021-11-14 23:25:57"
[1] "ABM train_candidate run end time : 2021-11-14 23:27:57"
[1] 3
[1] "ABM train_candidate run start time : 2021-11-14 23:27:57"
[1] "ABM train_candidate run end time : 2021-11-14 23:30:01"
[1] 4
[1] "ABM train_candidate run start time : 2021-11-14 23:30:02"
[1] "ABM train_candidate run end time : 2021-11-14 23:36:16"
[1] 5
[1] "ABM train_candidate run start time : 2021-11-14 23:36:16"
[1] "ABM train_candidate run end time : 2021-11-14 23:39:52"
[1] 6
[1] "ABM train_candidate run start time : 2021-11-14 23:39:52"
[1] "ABM train_candidate run end time : 2021-11-14 23:43:12"
[1] 7
[1] "ABM train_candidate run start time 

## Quit NL

In [22]:
NLQuit(nl.obj = nl.model)
#NLQuit(all=TRUE)